In [1]:
import haskellian.either as E
from haskellian.either import IsLeft
from pydantic import BaseModel, Field
from quicktype_ts import pydantic2typescript

TypeError: Too many arguments for <class 'haskellian.either.type.Left'>; actual 2, expected 1

'{"$defs": {"Left_int_": {"properties": {"left": {"title": "Left", "type": "integer"}, "tag": {"const": "left", "default": "left", "title": "Tag"}}, "required": ["left"], "title": "Left", "type": "object"}, "Right_str_": {"properties": {"right": {"title": "Right", "type": "string"}, "tag": {"const": "right", "default": "right", "title": "Tag"}}, "required": ["right"], "title": "Right", "type": "object"}}, "properties": {"x": {"discriminator": {"mapping": {"left": "#/$defs/Left_int_", "right": "#/$defs/Right_str_"}, "propertyName": "tag"}, "oneOf": [{"$ref": "#/$defs/Left_int_"}, {"$ref": "#/$defs/Right_str_"}], "title": "X"}}, "required": ["x"], "title": "Model", "type": "object"}'